In [1]:
import json
import os
from groq import Groq
import instructor
from pydantic import BaseModel, Field
import yfinance as yf
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

In [2]:
client = Groq(api_key=api_key)
model = "llama-3.3-70b-versatile"

In [3]:
tool_schema = {
    "name": "get_company_info",
    "description": "Get structured information about a company, including its full name, stock ticker, nationality, and stock prices.",
    "parameters": {
        "type": "object",
        "properties": {
            "company_name": {
                "type": "string",
                "description": (
                    "The full or commonly known name of the company (e.g., Apple, Tesla, IBM). "
                    "This will be used to fetch official data such as ticker, full legal name, and nationality."
                )
            }
        },
        "required": ["company_name"]
    }
}

In [4]:
class CompanyInfo(BaseModel):
    company_name: str = Field(description="The full legal name of the company")
    ticker: str = Field(description="The company ticker sybmbol name")
    real_price_usd: float = Field(default=None, description="The company stock price in USD")
    real_price_try: float = Field(default=None, description="The company stock price in TRY")
    nationality: str = Field(description="The company nationality")

class ResponseModel(BaseModel):
    tool_calls: list[CompanyInfo]

client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

In [5]:
def stock(stock_name):
    """
    Fetches the real-time stock price for a given stock name using yfinance.

    Args:
        stock_name: The stock ticker symbol (e.g., "AAPL", "MSFT").

    Returns:
        The current stock price as a float, or None if an error occurs.
    """
    try:
        ticker = yf.Ticker(stock_name)
        data = ticker.info
        return data.get("currentPrice")
    except Exception as e:
        print(f"Error fetching stock price for {stock_name}: {e}")
        return None

def get_price_info(ticker: str) -> dict:
    """
    Fetches the stock price in both USD and TRY using stock function and yfinance.
    """
    try:
        price_usd = stock(ticker)
        
        usd_try_rate = yf.Ticker("USDTRY=X").history(period="1d")["Close"].iloc[-1]
        
        price_try = round(price_usd * usd_try_rate, 2) if price_usd else None
        
        return {"price_usd": price_usd, "price_try": price_try}
    except Exception as e:
        print(f"Error fetching price for {ticker}: {e}")
        return {"price_usd": None, "price_try": None}

In [6]:
def run_conversation(user_prompt):
    messages = [
        {
            "role": "system",
            "content": f"You are an assistant that can use tools. You have access to the following tool: {tool_schema}"
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    response = client.chat.completions.create(
        model= model,
        response_model=ResponseModel,
        messages=messages,
        temperature=0.5,
        max_completion_tokens=1000,
    )

    tool_calls = response.tool_calls
    
    for call in tool_calls:
        price_info = get_price_info(call.ticker)
        call.real_price_usd = price_info["price_usd"]
        call.real_price_try = price_info["price_try"]
    
    return tool_calls

In [7]:
def build_df(tool_calls):
    tool_calls_dict = [
        {
            "company_name": call.company_name,
            "ticker": call.ticker,
            "real_price(USD)": call.real_price_usd,
            "real_price(TRY)": call.real_price_try,
            "nationality": call.nationality
        }
        for call in tool_calls
    ]
    
    df = pd.DataFrame(tool_calls_dict)
    
    return df

In [9]:
companies1 = "Give me structured information about the following companies: TSMC, Hepsiburada, MercadoLibre, Meta, HDFC Bank, and Tesla. "
tool_calls = run_conversation(companies1)

df = build_df(tool_calls)

df

,company_name,ticker,real_price(USD),real_price(TRY),nationality
0,Taiwan Semiconductor Manufacturing Company Lim...,TSM,176.40,6803.71,Taiwanese
1,Hepsiburada,HEPS,2.58,99.51,Turkish
2,"MercadoLibre, Inc.",MELI,2246.54,86648.60,Argentine
3,"Meta Platforms, Inc.",META,599.27,23113.72,American
4,HDFC Bank Limited,HDB,73.58,2837.97,Indian
5,"Tesla, Inc.",TSLA,280.26,10809.57,American


In [10]:
companies2 = "Give me structured information about the following companies:MercadoLibre, Meta, HDFC Bank, and Tesla. "
tool_calls = run_conversation(companies2)

df = build_df(tool_calls)

df

,company_name,ticker,real_price(USD),real_price(TRY),nationality
0,MercadoLibre Inc.,MELI,2246.54,86648.60,Argentine
1,Meta Platforms Inc.,META,599.27,23113.72,American
2,HDFC Bank Limited,HDB,73.58,2837.97,Indian
3,Tesla Inc.,TSLA,280.26,10809.57,American


In [11]:
companies3 = "Give me structured information about the following companies: Mercadlibre and IBM"
tool_calls = run_conversation(companies3)

df = build_df(tool_calls)

df

,company_name,ticker,real_price(USD),real_price(TRY),nationality
0,Mercadolibre Inc,MELI,2246.54,86648.60,Argentine
1,International Business Machines Corporation,IBM,249.18,9610.82,American
